Questions: 

1) "from func import..." cal_buoyancy and sounding_cal from other .py file, how does python know what "from func" is? 

2) what data am i using for this? what is the time range? is "fname='relative_height_data_2011-2014.nc'" a single .nc file or multiple files in one i.e. 2011-2014?

3) why are cal_buoyancy and sounding_cal called twice? difference between cal_buoyancy called is "h_profile", seems like different variable names saved after calling sound (lfc vs lfc_l)

4) don't understand "if __name__=="__main__": part and if i need it here

5) what is the "da" variable and how is it being used? why are we writing to it? da.close() at end indicates writing to this file in this script

8) why are there cape0 and cape1 and cin0 and cin1? 



In [9]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  6 16:05:07 2019
class of buoyancy calculate
@author: gloriabruin
"""

# -*- coding: utf-8 -*-
#'''calculate buoyancy from sounding'''
import numpy as np
import sys
from metpy.calc import (dewpoint_from_specific_humidity,
                        dewpoint_from_relative_humidity,
                        mixing_ratio_from_relative_humidity,
                        relative_humidity_from_dewpoint,
                        find_intersections,el,#lfc,
                        parcel_profile,                        
                        cape_cin,lcl)#,relative_humidity_from_mixing_ratio,mixing_ratio_from_relative_humidity
#from metpy.calc import parcel_profile,lcl

from func import cal_buoyancy,sounding_cal
from metpy.units import units
import datetime as dt
import matplotlib.pyplot as plt
import os
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from netCDF4 import Dataset

In [ ]:
datadir='/home/gloriabruin/LAC/vars_from_sgpsounde'
time_standard = dt.datetime(1970,1,1,0,0)   

class result_struct():
    def __init__(self):
        self.cape0=[]
        self.cin0 =[]
        self.buoyance_a0=[]
        self.buoyance_b0=[]
        self.entrain  = []
        self.BP0      = []
        self.TP0      = []
        self.QP0      = []
        self.cape1=[]
        self.cin1 =[]
        self.buoyance_a1=[]
        self.buoyance_b1=[]
        self.BP1      = []
        self.TP1      = []
        self.QP1      = []
        
if __name__=="__main__":
    entrain=2 #DIA
    plot_index=0 #1:plot 0:no plot
    
    fname='relative_height_data_2011-2014.nc'
    season_name_list =['MAM','JJA','SON','DJF']
    da =  Dataset(os.path.join(datadir,'integrated_buoyancy.nc'),'w')
    for season_name in season_name_list:
      data = Dataset(os.path.join(datadir,fname),'r')
    #season_name='SON'
      tt = data[season_name]['time']
      # time=[]
      # for i in tt:
      #     time.append(time_standard + dt.timedelta(seconds=int(i)))    
      t    = data[season_name]['temperature'][:]
      dp   = data[season_name]['dp'][:]
      alt  = data[season_name]['alt'][:]
      p    = data[season_name]['press'][:]
      p_lt = data[season_name]['press_lt'][:]
      s_theta = data[season_name]['s_theta'][:]
      sh      = data[season_name]['sh'][:]
      slope   = data[season_name]['slope'][:]
      interp  = data[season_name]['interp'][:]
      r       = data[season_name]['R'][:]
      p_value = data[season_name]['P_value'][:]
      r_sh    = data[season_name]['r_sh'][:]
      dp_temp = dp.copy()
      # dp_lt=[]
      # r_dp_lt=[]
      surf_index=[]
      surf_p=[]
      for i in range(len(sh)):
          surf_index = np.append(surf_index,np.where(~np.isnan(alt[i]))[0][0])
          surf_p = np.append(surf_p,p[int(surf_index[-1])])
          # dp_lt = np.append(dp_lt,dewpoint_from_specific_humidity(sh[i,:], t[i,int(surf_index[-1]):int( surf_index[-1]+sh.shape[1])]*units('degC'),surf_p[-1] - p_lt*units('hPa')))
          # r_dp_lt = np.append( r_dp_lt, dewpoint_from_specific_humidity(r_sh[i,:], t[i,int(surf_index[-1]):int( surf_index[-1]+sh.shape[1])]*units('degC'),surf_p[-1] -  p_lt*units('hPa')))
          dp_temp[i, int(surf_index[-1]):int( surf_index[-1]+300)]=dewpoint_from_specific_humidity(r_sh[i,:], t[i,int(surf_index[-1]):int( surf_index[-1]+sh.shape[1])]*units('degC'),surf_p[-1] - p_lt*units('hPa'))
      # dp_lt = np.reshape(dp_lt,(-1,len(p_lt)))
      # r_dp_lt = np.reshape(r_dp_lt,(-1,len(p_lt)))
        
    #finding indices where r_sh is less than zero and saving those indices to remove from other variables
      delete_index = np.unique(np.where(r_sh<0)[0]).astype('int')
      t       = np.delete(t,delete_index,0)
      dp      = np.delete(dp,delete_index,0)
      alt     = np.delete(alt,delete_index,0)
      s_theta = np.delete(s_theta,delete_index)
      sh      = np.delete(sh     ,delete_index,0)
    #    slope   = np.delete(slope  ,delete_index)
    #    interp  = np.delete(interp ,delete_index)
    #    r       = np.delete(r      ,delete_index)
    #    p_value = np.delete(p_value,delete_index)
      r_sh    = np.delete(r_sh   ,delete_index,0)
      dp_temp = np.delete(dp_temp,delete_index,0)
      tt      = np.delete(tt,delete_index)
      surf_p  = np.delete(surf_p,delete_index)
      surf_index = np.delete(surf_index,delete_index)
  #    time,press,alt,dp,temperature,press_lt,s_theta,sh_i=load_SGP()
      # inddd=np.array([  50.,  350.,  525.,  700.,  925.,  975., 1025.]).astype('int')
      # inddd=[]
  #    dp_temp2 = dp.copy()
      results_DIA   = result_struct()
      results_const = result_struct()
      lfc     =[]
      lfc_l   =[]
      el      =[]
      el_l    =[]
      lfc_ept     =[]
      lfc_l_ept   =[]
      el_ept      =[]
      el_l_ept    =[]
     # tt  = tt[:10]
      for i in range(0,len(tt)):
      #    print(i)
      # for i in range(1):
      #     i=1202
          # print("CALCULATE SETTING:")      
          # print('\tTIME:',time[i].strftime('%H:%M,%b-%d,%Y'))
          # print('\tLOCATION: SGP(36° 36′ N, 97° 29′  W)')
      

          data = cal_buoyancy(   t_profile = t[i,:],         # temperature profile
                          h_profile = dp[i,:],         # humidigy profile-units:g/kg
                          p_profile = p,         # units:hPa
                          alt_profile = alt[i,:],
                          plot_index  =plot_index,
  #                        td_r_q   = 1,      #1 for doewpoint; 2 for mixing ratio;3 for relative humidity ; 4 for sh
  #                        source   = 1,      # source of parcel
  #                        z_opt    = 0,      # height is calculated, if not given
  #                        z0       = 0,      # surface hegith
  #                        zs       = 0,      # parcel start height/presure,depends on z_opt
  #                        adiabat  = 1,      # Formulation of moist adiabat
                          entrain_opt = entrain,   # entrainment option 1：const 2:DIA
  #                        ce       = 0.4 ,    # parameter related to calculation of entrainment rate
  #                        x_en     = 0.001,   # constant entrainment rate
  #                        prc_w    = 0.0005 , # percentage per m (0-1) of condensate kept in the parcel
  #                        qc_max   = 0.001,   # max mixing ratio of condensate in cloud
  #                        ice      = 1  ,   # include ice process
  #                        tice     = 233.15 , # temperature at which all condensate will be ice form
  #                        tp_te    = 0  ,    # option to include the coefficient tp/te in some calculation                           
  #                        newton_step_dt   = 10,
  #                        switch_off_ent_t = 0,
  #                        switch_off_ent_r = 0,
  #                        index=1
                          )
          sound =  sounding_cal(data.p_in[data.ind]*units('Pa'), 
                             data.t_in[data.ind]*units('K'), 
                             data.td_in[data.ind]*units('K'), 
                             parcel_temperature_profile=data.tp[data.ind]*units('K'),
                             specific_pressure_start = (surf_p[i]-300)*units('hPa'),
                             specific_pressure_end = 100*units('hPa'))
          lfc.append(sound.lfc.p.magnitude)
          el.append(sound.el.p.magnitude)
          lfc_ept.append(sound.lfc.ept.magnitude)
          el_ept.append(sound.el.ept.magnitude)
          
            #results_DIA from result_struct()
          results_DIA.cape0 = np.append(results_DIA.cape0,sound.p_buoyancy)
          results_DIA.cin0  = np.append(results_DIA.cin0, sound.n_buoyancy)
          results_DIA.buoyance_a0 = np.append(results_DIA.buoyance_a0,sound.buoyancy_above)
          results_DIA.buoyance_b0 = np.append(results_DIA.buoyance_b0,sound.buoyancy_below)
          results_DIA.entrain  = np.append(results_DIA.entrain,data.X)
          results_DIA.BP0      = np.append(results_DIA.BP0,data.bp)
          results_DIA.TP0      = np.append(results_DIA.TP0,data.tp)
          results_DIA.QP0      = np.append(results_DIA.QP0,data.qp)
          #lfc= np.append(lfc,sound.lfc.p)
          data = cal_buoyancy(   t_profile = t[i,:],         # temperature profile
                          h_profile = dp_temp[i,:],         # humidigy profile-units:g/kg
                          p_profile = p,         # units:hPa
                          alt_profile = alt[i,:],
                          plot_index  =plot_index,
  #                        td_r_q   = 1,      #1 for doewpoint; 2 for mixing ratio;3 for relative humidity ; 4 for sh
  #                        source   = 1,      # source of parcel
  #                        z_opt    = 0,      # height is calculated, if not given
  #                        z0       = 0,      # surface hegith
  #                        zs       = 0,      # parcel start height/presure,depends on z_opt
  #                        adiabat  = 1,      # Formulation of moist adiabat
                          entrain_opt = entrain,   # entrainment option
  #                        ce       = 0.4 ,    # parameter related to calculation of entrainment rate
  #                        x_en     = 0.001,   # constant entrainment rate
  #                        prc_w    = 0.0005 , # percentage per m (0-1) of condensate kept in the parcel
  #                        qc_max   = 0.001,   # max mixing ratio of condensate in cloud
  #                        ice      = 1  ,   # include ice process
  #                        tice     = 233.15 , # temperature at which all condensate will be ice form
  #                        tp_te    = 0  ,    # option to include the coefficient tp/te in some calculation                           
  #                        newton_step_dt   = 10,
  #                        switch_off_ent_t = 0,
  #                        switch_off_ent_r = 0,
  #                        index=1
                          )
         
          sound =  sounding_cal(data.p_in[data.ind]*units('Pa'), 
                             data.t_in[data.ind]*units('K'), 
                             data.td_in[data.ind]*units('K'), 
                             parcel_temperature_profile=data.tp[data.ind]*units('K'),
                             specific_pressure_start = (surf_p[i]-300)*units('hPa'),
                             specific_pressure_end = 100*units('hPa'))
          lfc_l.append(sound.lfc.p.magnitude)
          el_l.append(sound.el.p.magnitude)
          lfc_l_ept.append(sound.lfc.ept.magnitude)
          el_l_ept.append(sound.el.ept.magnitude)
            
          results_DIA.cape1 = np.append(results_DIA.cape1,sound.p_buoyancy)
          results_DIA.cin1  = np.append(results_DIA.cin1, sound.n_buoyancy)
          results_DIA.buoyance_a1 = np.append(results_DIA.buoyance_a1,sound.buoyancy_above)
          results_DIA.buoyance_b1 = np.append(results_DIA.buoyance_b1,sound.buoyancy_below)
          
         
          results_DIA.BP1      = np.append(results_DIA.BP1     ,data.bp)
          results_DIA.TP1      = np.append(results_DIA.TP1     ,data.tp)
          results_DIA.QP1      = np.append(results_DIA.QP1     ,data.qp)
      
      results_DIA.entrain  = np.reshape(results_DIA.entrain,(len(tt),-1))
      results_DIA.BP0      = np.reshape(results_DIA.BP0,(len(tt),-1))
      results_DIA.TP0      = np.reshape(results_DIA.TP0,(len(tt),-1))
      results_DIA.QP0      = np.reshape(results_DIA.QP0,(len(tt),-1))
    
      results_DIA.BP1      = np.reshape(results_DIA.BP1,(len(tt),-1))
      results_DIA.TP1      = np.reshape(results_DIA.TP1,(len(tt),-1))
      results_DIA.QP1      = np.reshape(results_DIA.QP1,(len(tt),-1)) 
  
      g=da.createGroup('/%s'%season_name)
      g.createDimension('time',len(tt))
      g.createDimension('press',len(p))
      g.createDimension('press_lt',len(p_lt))

        #writing to da file by creating variables as listed below that were created in above code
      d = da.createVariable('/%s/time'%season_name,'i',('time',))
      d.units = 'seconds from 1970-1-1-0-0'
      d[:] = tt
      d = da.createVariable('/%s/press'%season_name,'f',('press',))
      d.units = 'hPa'
      d[:]=p
      d = da.createVariable('/%s/temperature'%season_name,'f',('time','press'))
      d.description='Temperature profile'
      d.units = 'degC'
      d[:]=t
      d = da.createVariable('/%s/dp'%season_name,'f',('time','press'))
      d.units = 'degC'
      d[:]=dp
      d = da.createVariable('/%s/dp_l'%season_name,'f',('time','press'))
      d.description = 'dewpoint with LT reconstucted by statiscially regress'
      d.units = 'degC'
      d[:]=dp_temp
      d = da.createVariable('/%s/alt'%season_name,'f',('time','press',))
      d.description='Altitude from sea level'
      d.units = 'm'
      d[:]=alt
      d = da.createVariable('/%s/press_lt'%season_name,'f',('press_lt',))
      d.description='LT regular pressure levels difference from surface'
      d.units = 'hPa'
      d[:]=p_lt
      d = da.createVariable('/%s/s_theta'%season_name,'f',('time',))
      d.description='Surface Equivalent potential temperature(K)'
      d.units = 'K'
      d[:]=s_theta
      d = da.createVariable('/%s/surf_index'%season_name,'f',('time',))
      d.description='Surface index in array'
      d.units = 'hPa'
      d[:]=surf_index
      d = da.createVariable('/%s/sh'%season_name,'f',('time','press_lt'))
      d.units = 'g/kg'
      d[:]=sh
      d = da.createVariable('/%s/slope'%season_name,'f',('press_lt',))
      d[:]=slope
      d = da.createVariable('/%s/interp'%season_name,'f',('press_lt',))
      d[:]=interp
      d = da.createVariable('/%s/R'%season_name,'f',('press_lt',))
      d[:]=r
      d = da.createVariable('/%s/P_value'%season_name,'f',('press_lt',))
      d[:]=p_value
      d = da.createVariable('/%s/r_sh'%season_name,'f',('time','press_lt',))
      d.description='Reconstructed Specific Humidity'
      d[:]=r_sh

      d = da.createVariable('/%s/entrain'%season_name,'f',('time','press',))
      d.description='DIA entrain'
      d[:]=results_DIA.entrain
      d = da.createVariable('/%s/bp'%season_name,'f',('time','press',))
      d.description='Total parcel buoyancy(m/s2)'
      d[:]=results_DIA.BP0
      d = da.createVariable('/%s/tp'%season_name,'f',('time','press',))
      d.description='Total parcel temperature profile'
      d[:]=results_DIA.TP0
      # d = da.createVariable('/%s/qp'%season_name,'f',('time','press',))
      # d.description='Total parcel mixing ratio of water vapor(g/kg)'
      # d[:]=results_DIA.QP0
      d = da.createVariable('/%s/cape'%season_name,'f',('time',))
      d.description='Total parcel cape'
      d[:]=results_DIA.cape0
      d = da.createVariable('/%s/cin'%season_name,'f',('time',))
      d.description='Total parcel cin'
      d[:]=results_DIA.cin0
      d = da.createVariable('/%s/buoyance_a'%season_name,'f',('time',))
      d.description='Total parcel integrated buoyancy above 300hPa above surface'
      d[:]=results_DIA.buoyance_a0

      d = da.createVariable('/%s/buoyance_b'%season_name,'f',('time',))
      d.description='Total parcel integrated buoyancy below 300hPa above surface'
      d[:]=results_DIA.buoyance_b0
      
      d = da.createVariable('/%s/bp_l'%season_name,'f',('time','press',))
      d.description='land related parcel buoyancy(m/s2)'
      d[:]=results_DIA.BP1
      d = da.createVariable('/%s/tp_l'%season_name,'f',('time','press',))
      d.description='land related parcel temperature profile'
      d[:]=results_DIA.TP1
      # d = da.createVariable('/%s/qp_l'%season_name,'f',('time','press',))
      # d.description='land related parcel mixing ratio of water vapor(g/kg)'
      # d[:]=results_DIA.QP1
      d = da.createVariable('/%s/cape_l'%season_name,'f',('time',))
      d.description='land related  parcel cape'
      d[:]=results_DIA.cape1
      d = da.createVariable('/%s/cin_l'%season_name,'f',('time',))
      d.description='land related parcel cin'
      d[:]=results_DIA.cin1
      d = da.createVariable('/%s/buoyance_a_l'%season_name,'f',('time',))
      d.description='land related  parcel integrated buoyancy above 300hPa above surface'
      d[:]=results_DIA.buoyance_a1
      d = da.createVariable('/%s/buoyance_b_l'%season_name,'f',('time',))
      d.description='land related  parcel integrated buoyancy below 300hPa above surface'
      d[:]=results_DIA.buoyance_b1

      d = da.createVariable('/%s/lfc'%season_name,'f',('time',))
      d.description='Pressure of LFC(hPa)'
      d[:]=lfc
      d = da.createVariable('/%s/lfc_theta'%season_name,'f',('time',))
      d.description='Equivalent potential temperature of LFC(K)'
      d[:]=lfc_ept

      d = da.createVariable('/%s/lfc_l'%season_name,'f',('time',))
      d.description='Pressure of land related LFC(hPa)'
      d[:]=lfc_l
      d = da.createVariable('/%s/lfc_l_theta'%season_name,'f',('time',))
      d.description='Equivalent potential temperature of land related LFC(K)'
      d[:]=lfc_l_ept

      d = da.createVariable('/%s/el'%season_name,'f',('time',))
      d.description='Pressure of EL(hPa)'
      d[:]=el
      d = da.createVariable('/%s/el_theta'%season_name,'f',('time',))
      d.description='Equivalent potential temperature of EL(K)'
      d[:]=el_ept

      d = da.createVariable('/%s/el_l'%season_name,'f',('time',))
      d.description='Pressure of land related EL(hPa)'
      d[:]=el_l
      d = da.createVariable('/%s/el_l_theta'%season_name,'f',('time',))
      d.description='Equivalent potential temperature of land related EL(K)'
      d[:]=el_l_ept
      print('%s complete!'%season_name)
    da.close()

